In [1]:
from triple_stream_data_utils import plot_scatter_distribution, create_heatmap_histogram_from_lists
from tripleStreamsDataloaderUtils import load_compiled_dataset_pkl_bz2

Could not import fluidsynth. AUDIO rendering will not work.
/Users/bezha/anaconda3/envs/TripleStreams/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Make sure you've already ran python CompileStreamsAndFeatures.py
import os
accentThresh = 0.75

root_path = f"data/triple_streams/split_2bars/compiled_data/accentThresh{accentThresh}"
all_pkl_bz2_files = [f for f in os.listdir(root_path) if f.endswith('.pkl.bz2')]

# Available Datasets
available_datasets_path = {
    all_pkl_bz2_file.split(".pkl.bz2")[0]: os.path.join(root_path, all_pkl_bz2_file) for all_pkl_bz2_file in all_pkl_bz2_files
}
# dict_keys(['input_hvos', 'output_hvos', 'flat_out_hvos', 'sample_id', 'collection', 'metadata', 'qpm', 'Flat Out Vs. Input | Hits | Hamming', 'Flat Out Vs. Input | Accent | Jaccard', 'OS1_OF Jaccard', 'OS2_OF Jaccard', 'OS3_OF Jaccard', 'pearson', 'cosine', 'euclidean_similarity', 'mae_similarity', 'dtw_similarity'])

available_datasets_path,

({'ttd_both-is-and_both_flattened_left_right_Accent_thresh': 'data/triple_streams/split_2bars/compiled_data/accentThresh0.75/ttd_both-is-and_both_flattened_left_right_Accent_thresh.pkl.bz2',
  'ttd_both-is-or_both_flattened_left_right_Accent_thresh': 'data/triple_streams/split_2bars/compiled_data/accentThresh0.75/ttd_both-is-or_both_flattened_left_right_Accent_thresh.pkl.bz2',
  'groove_midi_groove_high_low_mid_Accent_thresh': 'data/triple_streams/split_2bars/compiled_data/accentThresh0.75/groove_midi_groove_high_low_mid_Accent_thresh.pkl.bz2'},)

In [3]:
from make_triple_stream_plots import build_plots

feat1 = "Flat Out Vs. Input | Hits | Hamming"
feat2 = "Flat Out Vs. Input | Accent | Jaccard"


os.makedirs(os.path.join(root_path, "plots"), exist_ok=True)
# dataset_tags = ["lmd_bass_brass_guitar_percussive", "elbg_both_flattened_left_right"]

dataset_tags = [path.split("/")[-1].split(".pkl.bz2")[0] for path in sorted(available_datasets_path)]
# find all pkl.bz2 files in the root_path

output_path = build_plots(
    root=root_path,
    dataset_tags=dataset_tags,
    feature1=feat1,
    feature2=feat2,
    n_bins_feat1=None,
    n_bins_feat2=10,
    scatter_max_points=100000,
    output_html_path=os.path.join(os.path.join(root_path, "plots"), f"<{feat1}>vs<{feat2}>.html"),
    force_recalc=False,
)

print(f"HTML saved at: {output_path}")

Loading datasets...


Datasets: 100%|██████████| 3/3 [00:15<00:00,  5.01s/it]


Preparing heatmap caches...


Heatmaps: 100%|██████████| 3/3 [00:00<00:00, 96.51it/s]


Preparing scatter data...


Scatter: 100%|██████████| 3/3 [00:03<00:00,  1.28s/it]


HTML saved at: /Users/bezha/PycharmProjects/TripleStreams/data/triple_streams/split_2bars/compiled_data/accentThresh0.75/plots/<Flat Out Vs. Input | Hits | Hamming>vs<Flat Out Vs. Input | Accent | Jaccard>.html


In [4]:
# npz_path = "/Users/bezha/PycharmProjects/TripleStreams/data/triple_streams/split_2bars/compiled_data/cache/hist2d__lmd_bass_brass_drum_guitar__Flat Out Vs. Input | Hits | Hamming__Flat Out Vs. Input | Accent | Jaccard__bins33x33.npz"
# # Load the npz file
# import numpy as np
# data = np.load(npz_path)
# data

In [5]:
data_ = load_compiled_dataset_pkl_bz2("data/triple_streams/split_2bars/compiled_data/accentThresh0.75/ttd_both-is-and_both_flattened_left_right_Accent_thresh.pkl.bz2")
create_heatmap_histogram_from_lists(
    feat1=data_[feat1],
    feat2=data_[feat2],
)

In [6]:
# sample_ix = 100 # for each sample 24 variations
# input_hvos = loaded_datasets["input_hvos"][sample_ix*24: ((sample_ix+1)*24)]
# output_hvos = loaded_datasets["output_hvos"][sample_ix*24: ((sample_ix+1)*24)]
# metadatas = loaded_datasets["metadata"][sample_ix*24: ((sample_ix+1)*24)]
# print(len(input_hvos), len(output_hvos), len(metadatas))
# print(loaded_datasets["Flat Out Vs. Input | Hits | Hamming"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["Flat Out Vs. Input | Accent | Jaccard"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["OS1_OF Jaccard"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["OS2_OF Jaccard"][sample_ix*24: ((sample_ix+1)*24)])
# print(loaded_datasets["OS3_OF Jaccard"][sample_ix*24: ((sample_ix+1)*24)])
# create_multitab_from_HVO_Sequences(compile_into_list_of_hvo_seqs(input_hvos, output_hvos, metadatas, qpms=None))